Setting seed

In [ ]:
import random

random.seed(10)

Importing tensorflow

In [ ]:
import tensorflow as tf

Installing the Hugging Face Library

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 13.0MB/s 
     |████████████████████████████████| 3.3MB 50.6MB/s 
     |████████████████████████████████| 870kB 46.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=8f336afab887dd45d5f5b8db9c11643ddc6b65c2fe0be3c4824db8ffa8e316ec
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


Loading Reddit Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
lav_path = 'gdrive/My Drive/W266Project_Lav_Shalz/train-balanced-sarcasm.csv'
shalz_path = '/content/gdrive/MyDrive/Colab Notebooks/train-balanced-sarcasm.csv'

import pandas as pd
full_df = pd.read_csv(lav_path)

Mounted at /content/gdrive


Viewing first few rows of Reddit Dataset

In [ ]:
full_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


Reporting the number of comments

In [ ]:
print('Number of comments: {:,}\n'.format(full_df.shape[0]))

Number of comments: 1,010,826



Looking at a sample of comments that are labeled as sarcastic

In [ ]:
full_df.loc[full_df.label == 1].sample(5)[['comment', 'label']]

,comment,label
726038,Now if we manage to convince Belarus to join w...,1
369534,"Ahh yes, because as everyone knows, Greek memb...",1
641350,and here i thought only repubs fearmonger,1
348605,This game is not for solo players!,1
943550,His stool fucking was off the chain,1


Looking at a sample of comments that are labeled as not sarcastic

In [ ]:
full_df.loc[full_df.label == 0].sample(5)[['comment', 'label']]

,comment,label
713063,"consoles still use physical media, and this re...",0
465999,"Haha, you fooled me with nostalgia :-) Well done",0
116768,Done and done,0
743458,People who are apathetic about an issue aren't...,0
699619,"Lol @ butthurt downvoters, mad that you insinu...",0


Removing missing values

In [ ]:
# remove na 
full_df = full_df[full_df['comment'].notna()]

Selecting a subset of the data

In [ ]:
# select a subset of the data
s0 = full_df.label[full_df.label.eq(0)].sample(5000).index
s1 = full_df.label[full_df.label.eq(1)].sample(5000).index 
df = full_df.loc[s0.union(s1)]
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
12,0,They have an agenda.,Readbooks6,exmormon,4,-1,-1,2016-10,2016-10-15 01:14:03,There's no time to worry about leaks when they...
236,0,I wish for a 13 inch pianist,hookbill2,AskReddit,3,-1,-1,2016-11,2016-11-22 13:47:28,You have one wish. But it'll only come true if...
439,0,"That's ""Old America "" talk... ""new amerika"" is...",therealhood,politics,2,-1,-1,2016-11,2016-11-30 16:57:46,"No, that is completely wrong: the American peo..."
557,0,I'm sensing some bitterness for being a fan of...,cmanson,nfl,4,-1,-1,2016-11,2016-11-01 21:43:01,Dallas is. Gb is over weight Wisconsinites team
681,1,"Ahh, the glorious american cuisine.",dr_obfuscation,oddlysatisfying,1,-1,-1,2016-11,2016-11-02 00:45:58,This grocery aisle
...,...,...,...,...,...,...,...,...,...,...
1010606,1,"Wouldn't that depend on your definition of ""is""?",cmotdibbler,atheism,1,1,0,2009-03,2009-03-13 13:46:24,I'd have to agree with (uhhhh what a username)...
1010678,1,The quality would be better if maybe they got ...,screechyd,funny,2,2,0,2009-10,2009-10-08 13:53:54,Their quality is steadily improving. I thought...
1010737,1,Ubuntu isn't modern like Windows 7.,dawnerd,AskReddit,2,2,0,2009-08,2009-08-27 01:25:48,Ubuntu does
1010767,1,Anti-semitic holocaust denier!,veritaze,Economics,5,5,0,2009-01,2009-01-31 03:32:41,Secret Israeli database reveals full extent of...


Extracting the comments and the labels of the dataset as numpy ndarrays

In [ ]:
import numpy as np

In [ ]:
comments = df.comment.values
labels = df.label.values

Tokenization

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Loading BERT tokenizer...


Apply the tokenizer to just one comment just to see the output.

In [ ]:
# Print the original comment.
print(' Original: ', comments[0])

# Print the comment split into tokens.
print('Tokenized: ', tokenizer.tokenize(comments[0]))

# Print the comment mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(comments[0])))

 Original:  They have an agenda.
Tokenized:  ['They', 'have', 'an', 'agenda', '.']
Token IDs:  [1220, 1138, 1126, 12932, 119]


# Tokenize and Split Data

## Tokenize dataset using batch encode

In [ ]:
batch_encoded_dict = tokenizer.batch_encode_plus(
                      comments,
                      add_special_tokens = True,
                      max_length = 30,
                      pad_to_max_length = True,
                      return_token_type_ids=True,
                      return_attention_mask = True,
                      return_tensors = 'tf'
                     )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
labels = tf.convert_to_tensor(labels)

In [ ]:
labels

<tf.Tensor: shape=(10000,), dtype=int64, numpy=array([0, 0, 0, ..., 1, 1, 1])>

## Combine the training inputs into a tensorflow dataset

In [ ]:
def example_to_features(input_ids, attention_masks, token_type_ids, y):
  return {"input_ids": input_ids,
          "token_type_ids": token_type_ids,
          "attention_mask": attention_masks}, y

dataset = tf.data.Dataset.from_tensor_slices((batch_encoded_dict['input_ids'], batch_encoded_dict['token_type_ids'], batch_encoded_dict['attention_mask'], 
                                              labels)).map(example_to_features)

## Split the tensorflow dataset into Training, Validation, Test

In [ ]:
# Create a 70%-15%-15% train-validation-test split
# Calculate the number of samples to include in each set
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = int(0.15 * dataset_size)
dataset = dataset.shuffle(dataset_size)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = dataset.skip(test_size)
test_dataset = dataset.take(test_size)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

7,000 training samples
1,500 validation samples
1,500 test samples


# Modeling with Various Hyperparameters

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

In [ ]:
config = BertConfig.from_pretrained('bert-base-cased')

Define function that takes a EvalPrediction and outputs metrics as a dictionary (string to value)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = labels.flatten()
    acc = np.sum(preds_flat == labels_flat) / len(labels_flat)
    return {'accuracy': acc}

## Training batch size 16, Evaluation batch size 64, Learning rate 5e-5, No. of Epochs 2, Weight decay 0.01, Trained on 7,000 examples, Validated on 1,500 examples, Tested on 1,500 examples, Hidden dropout prob 0.1



Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import transformers

transformers.logging.set_verbosity_info()

Train and Validate

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 7000
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 438
  Total optimization steps = 876


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

{'loss': 0.62576735, 'learning_rate': 5e-05, 'epoch': 1.1415525114155252, 'step': 500}
Saving checkpoint for step 500 at ./results/checkpoint/ckpt-1
Training took: 0:01:37.380342


Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 1500
  Num examples in used in evaluation = 1536
  Batch size = 64


{'eval_loss': 0.4483521382013957, 'eval_accuracy': 0.791015625, 'epoch': 2.0, 'step': 876}


{'eval_accuracy': 0.791015625, 'eval_loss': 0.4483521382013957}

Clear session

In [ ]:
tf.keras.backend.clear_session()